# SQL query from table names - Continued

In [57]:
from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

## The old Prompt

In [58]:
#The old prompt
old_context = [ {'role':'system', 'content':"""
you are a bot to assist in create SQL commands, all your answers should start with \
this is your SQL, and after that an SQL that can do what the user request. \
Your Database is composed by a SQL database with some tables. \
Try to maintain the SQL order simple.
Put the SQL command in white letters with a black background, and just after \
a simple and concise text explaining how it works.
If the user ask for something that can not be solved with an SQL Order \
just answer something nice and simple, maximum 10 words, asking him for something that \
can be solved with SQL.
"""} ]

old_context.append( {'role':'system', 'content':"""
first table:
{
  "tableName": "employees",
  "fields": [
    {
      "nombre": "ID_usr",
      "tipo": "int"
    },
    {
      "nombre": "name",
      "tipo": "varchar"
    }
  ]
}
"""
})

old_context.append( {'role':'system', 'content':"""
second table:
{
  "tableName": "salary",
  "fields": [
    {
      "nombre": "ID_usr",
      "type": "int"
    },
    {
      "name": "year",
      "type": "date"
    },
    {
      "name": "salary",
      "type": "float"
    }
  ]
}
"""
})

old_context.append( {'role':'system', 'content':"""
third table:
{
  "tablename": "studies",
  "fields": [
    {
      "name": "ID",
      "type": "int"
    },
    {
      "name": "ID_usr",
      "type": "int"
    },
    {
      "name": "educational_level",
      "type": "int"
    },
    {
      "name": "Institution",
      "type": "varchar"
    },
    {
      "name": "Years",
      "type": "date"
    }
    {
      "name": "Speciality",
      "type": "varchar"
    }
  ]
}
"""
})

## New Prompt.
We are going to improve it following the instructions of a Paper from the Ohaio University: [How to Prompt LLMs for Text-to-SQL: A Study in Zero-shot, Single-domain, and Cross-domain Settings](https://arxiv.org/abs/2305.11853). I recommend you read that paper.

For each table, we will define the structure using the same syntax as in a SQL create table command, and add the sample rows of the content.

Finally, at the end of the prompt, we'll include some example queries with the SQL that the model should generate. This technique is called Few-Shot Samples, in which we provide the prompt with some examples to assist it in generating the correct SQL.


In [59]:
context = [
    {
        'role': 'system',
        'content': "You are a helpful assistant that explains SQL schemas."
    },
    {
        'role': 'user',
        'content': '''
CREATE TABLE customers (
    customer_id SERIAL PRIMARY KEY,
    name VARCHAR(100),
    email VARCHAR(100)
);

CREATE TABLE products (
    product_id SERIAL PRIMARY KEY,
    name VARCHAR(100),
    price DECIMAL(10, 2)
);

CREATE TABLE orders (
    order_id SERIAL PRIMARY KEY,
    customer_id INT REFERENCES customers(customer_id),
    order_date DATE
);
'''
    }
]


In [60]:
#FEW SHOT SAMPLES
context.append( {'role':'system', 'content':"""
 -- Maintain the SQL order simple and efficient as you can, using valid SQL Lite, answer the following questions for the table provided above.
WRITE IN YOUR CONTEXT QUERIES HERE
"""
})

In [61]:
#Functio to call the model.
def return_CCRMSQL(user_message, context):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)

    newcontext = context.copy()
    newcontext.append({'role':'user', 'content':"question: " + user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=newcontext,
            temperature=0,
        )

    return (response.choices[0].message.content)

## NL2SQL Samples
We're going to review some examples generated with the old prompt and others with the new prompt.

In [62]:
#new
context_user = context.copy()
print(return_CCRMSQL("""Which customers ordered a product named 'Laptop' in January 2024?""", context_user))

```sql
SELECT c.name
FROM customers c
JOIN orders o ON c.customer_id = o.customer_id
JOIN order_details od ON o.order_id = od.order_id
JOIN products p ON od.product_id = p.product_id
WHERE p.name = 'Laptop' AND o.order_date >= '2024-01-01' AND o.order_date <= '2024-01-31';
```


In [63]:
#old
old_context_user = old_context.copy()
print(return_CCRMSQL("Which customers ordered a product named 'Laptop' in January 2024?", old_context_user))

This is your SQL:
```sql
SELECT e.name
FROM employees e
JOIN orders o ON e.ID_usr = o.ID_usr
JOIN products p ON o.product_id = p.product_id
WHERE p.product_name = 'Laptop' AND o.order_date BETWEEN '2024-01-01' AND '2024-01-31';
```

This SQL query retrieves the names of customers who ordered a product named 'Laptop' in January 2024 by joining the employees, orders, and products tables and filtering the results based on the product name and order date.


In [64]:
#new
print(return_CCRMSQL("Which customers ordered a product named 'Laptop' in January 2024?", context_user))

```sql
SELECT c.name
FROM customers c
JOIN orders o ON c.customer_id = o.customer_id
JOIN order_details od ON o.order_id = od.order_id
JOIN products p ON od.product_id = p.product_id
WHERE p.name = 'Laptop' AND o.order_date >= '2024-01-01' AND o.order_date <= '2024-01-31';
```


In [65]:
#old
print(return_CCRMSQL("Which customers ordered a product named 'Laptop' in January 2024?", old_context_user))

This is your SQL:
```sql
SELECT e.name
FROM employees e
JOIN orders o ON e.ID_usr = o.ID_usr
JOIN products p ON o.product_id = p.product_id
WHERE p.product_name = 'Laptop'
AND o.order_date BETWEEN '2024-01-01' AND '2024-01-31';
```

This SQL query retrieves the names of customers who ordered a product named 'Laptop' in January 2024 by joining the employees, orders, and products tables and filtering the results based on the product name and order date.


# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong.
     - What did you learn?

In [67]:
pqt1 = prompt_question_tables.format(
    tables=text_tables, 
    question="How many customers ordered products in January?"
)
print(return_CCRMSQL(pqt1, context_user))

To find out how many customers ordered products in January, you can use the following SQL query:

```sql
SELECT COUNT(DISTINCT customer_id) AS total_customers
FROM orders
WHERE EXTRACT(MONTH FROM order_date) = 1;
```

This query will count the distinct customer IDs from the `orders` table where the `order_date` is in January (month number 1).


In [68]:
pqt2 = prompt_question_tables.format(
    tables=text_tables,
    question="What is the total sales value for each product category?"
)
print(return_CCRMSQL(pqt2, context_user))

To calculate the total sales value for each product category, you would need to join the `products` and `orders` tables based on the product_id and then sum the total sales value for each product category. Here is the SQL query to achieve this:

```sql
SELECT p.name AS product_name, SUM(p.price) AS total_sales_value
FROM products p
JOIN order_details od ON p.product_id = od.product_id
JOIN orders o ON od.order_id = o.order_id
GROUP BY p.name;
```

In this query:
- We select the product name from the `products` table and calculate the total sales value by summing the price of each product.
- We join the `products` table with the `order_details` table on the product_id and then join the `order_details` table with the `orders` table on the order_id.
- Finally, we group the results by product name to get the total sales value for each product category.


In [ ]:
#3 a table with possibility to hallucinate extra tables - test
pqt3 = prompt_question_tables.format(
    tables=text_tables,
    question="Which customers have never placed an order?"
)
print(return_CCRMSQL(pqt3, context_user))

```sql
SELECT c.name
FROM customers c
LEFT JOIN orders o ON c.customer_id = o.customer_id
WHERE o.customer_id IS NULL;
```


###REPORT

In this case two first prompts were created correctly, where the third one was meant to test about hallucinating about non-existing tables.

Findings:
- prompt quality impacts on model accuracy = clear and structured prompts lead to focused queries
- broad instructions might lead to hallucinations
- when table names are limited and well-defined, the model performs better 
- without constrains, the model created a non-existing table 'order_details'
- table definitions do not have to be very detailed, it is enough for the model to understand schema roles
- reusable prompt templates seems to be more efficient